# Capstone - Final Project
## Author - Salinee Kingbaisomboon
### UW NetID: 1950831

Milestone 3 continues your work with the diaper manufacturing problem, using the same datasets:
1. A dataset file SECOM containing 1567 examples, each with 591 features, presented in a 1567 x 591 matrix
2. A labels file listing the classifications and date time stamp for each example 

## Instructions
1. Build a simple neural networks model
2. Build a DNN model
3. Build a RNN model
4. Summarize your findings with examples.  Explain what the manufacturer should focus on to optimize the diaper manufacturing process
5. Solicit specific feedback on your code

In [3]:
# Load necessary libraries
import pandas as pd
import numpy as np

from collections import Counter

from imblearn.over_sampling import SMOTE

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE # Recursive Feature Elimination (for backward model selection)
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, GRU

import warnings

warnings.filterwarnings("ignore") # To suppress warning

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Declare Functions used in this milestone

In [4]:
# Function to replace missing numeric values
def replace_missing_value(x, col):
    # We determine the locations of NaN
    HasNan = np.isnan(x.loc[:, col]) 
    # If there are NaN
    if sum(HasNan) > 0:
        # Calculate the Median for current column without Nan
        Median = np.nanmedian(x.loc[:, col])
        # Replace the missing value with Median
        x.loc[HasNan, col] = Median
    return x.loc[:, col]

In [5]:
# function that returns a tensorflow.data.Dataset object
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

# Read, merge and perform data cleaning

In [6]:
# Load data and label
dataFilename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
data_df = pd.read_csv(dataFilename, header=None, delim_whitespace=True)

labelFilename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'
label_df = pd.read_csv(labelFilename, header=None, delim_whitespace=True)

In [7]:
# Set pseudo names for input data
for col in data_df.columns:
    data_df.rename(columns = {col:'feature_'+ str(col)}, inplace = True)
    
# Set pseudo name for label data
label_df.rename(columns = {0:'HouseLineTestingResult'}, inplace = True)
label_df.rename(columns = {1:'TimeStamp'}, inplace = True)

# Since we didn't do the time series analysis, we don't need the timestamp column
# Drop the unused columns
label_df.drop(['TimeStamp'],axis=1, inplace=True)

In [8]:
# Clean input data
for col in data_df.columns:
    # Replace missing values with Median for current column without Nan
    data_df.loc[:, col] = replace_missing_value(data_df, col)

In [9]:
# Merge input and label data into one
df = pd.concat([data_df,label_df],axis=1)

In [10]:
# Normalize data frame
cols = df.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(np_scaled, columns = cols)

## Explore Data

In [11]:
# View first 5 rows
df_normalized.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,featu

In [12]:
print(df_normalized.shape) # Print DataFrame's size
print(df_normalized.dtypes) # Print DataFrame's data types

(1567, 591)
feature_0                 float64
feature_1                 float64
feature_2                 float64
feature_3                 float64
feature_4                 float64
feature_5                 float64
feature_6                 float64
feature_7                 float64
feature_8                 float64
feature_9                 float64
feature_10                float64
feature_11                float64
feature_12                float64
feature_13                float64
feature_14                float64
feature_15                float64
feature_16                float64
feature_17                float64
feature_18                float64
feature_19                float64
feature_20                float64
feature_21                float64
feature_22                float64
feature_23                float64
feature_24                float64
feature_25                float64
feature_26                float64
feature_27                float64
feature_28                float64
fe

In [13]:
# View Statistic Summary
df_normalized.describe()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,featu

## Handle Class Imbalance

In [14]:
# Extract features and target variable from data frame
targetOutcome = pd.DataFrame(df_normalized,columns=['HouseLineTestingResult'])
allInputs = pd.DataFrame(df_normalized,columns=df.columns.difference(['HouseLineTestingResult']))

In [15]:
# Modify data by handling class imbalance using SMOTE method
print('Original training targets shape {}'.format(Counter(targetOutcome.values.ravel())))
sm = SMOTE(random_state=0)
allInputs_resample, targetOutcome_resample = sm.fit_sample(allInputs, targetOutcome.values.ravel())
print('Resampled dataset shape {}'.format(Counter(targetOutcome_resample)))

Original training targets shape Counter({0.0: 1463, 1.0: 104})
Resampled dataset shape Counter({0.0: 1463, 1.0: 1463})


## Feature Selection

In [16]:
# Create linear regression model as the estimator
estimator = LinearRegression()
# From the Filter method, we select 10 features -- this number is arbitary (need to find the optimal number with further analysis)
selector = RFE(estimator, 10, step=1)
# Learn from this dataset
selector = selector.fit(allInputs, targetOutcome)
# The mask of selected features (which variables are selected)
print(selector.support_)
# Selected features are ranked 1. The variable with the highest rank is the one that is removed first
print(selector.ranking_)
# Print the name of selected features
f = selector.get_support(True) # the most important features
print('\033[1m\033[4mSeledted features from Backward Stepwise Model Selection are: \033[0m')
for f_index in f:
    print(df.columns[f_index])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False False False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False  T

In [17]:
# Create a new copy of data frame with selected features
selected_feature_names = ['feature_47','feature_60','feature_162','feature_164','feature_171','feature_196','feature_209','feature_324','feature_464','feature_466']
allInputs_feature_selected = allInputs[selected_feature_names].copy()

In [18]:
# View first 5 rows
allInputs_feature_selected.head()

,feature_47,feature_60,feature_162,feature_164,feature_171,feature_196,feature_209,feature_324,feature_464,feature_466
0,0.798994,0.533284,0.009981,0.026417,0.21645,0.009205,0.0,0.317877,0.0,0.0
1,0.780757,0.555275,0.088877,0.063291,0.37683,0.005046,0.0,0.430930,0.0,0.0
2,0.791108,0.810180,0.056955,0.064392,0.05772,0.008469,0.0,0.277554,0.0,0.0
3,0.794558,0.747554,0.081473,0.059439,0.05504,0.010574,0.0,0.272055,0.0,0.0
4,0.529771,0.617807,0.039136,0.022014,0.11173,0.009560,0.0,0.420716,0.0,0.0


In [19]:
# Split test(90%) and train (10%) data
X, XX, Y, YY = train_test_split(allInputs_feature_selected, targetOutcome, test_size = 0.1, random_state = 0)

In [20]:
print('Train: X=%s, Y=%s' % (X.shape, Y.shape))
print('Test: X=%s, Y=%s' % (XX.shape, YY.shape))

Train: X=(1410, 10), Y=(1410, 1)
Test: X=(157, 10), Y=(157, 1)


# Build Models

## 1. Simple Neural Networks model

In [21]:
# Use multilayer perceptron (MLP) from sklearn with solver = stochastic gradient descent
mlp = MLPClassifier(hidden_layer_sizes=(10),solver='sgd',learning_rate_init= 0.01, max_iter=500)

In [22]:
# Train the model
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=10, learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [23]:
# Evaluate model
scores = mlp.score(XX,YY)
print("Accuracy: %.2f%%" % (scores*100))

Accuracy: 97.45%


## 2. Deep Neural Networks model (DNN)

In [24]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in selected_feature_names:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [25]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 2 classes (Success or Failed)
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\salin\\AppData\\Local\\Temp\\tmprw1915lq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(X, Y, training=True),
    steps=500)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

In [27]:
# Evaluate model
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(XX, YY, training=False))
scores = eval_result['accuracy']
print("Accuracy: %.2f%%" % (scores*100))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-15T22:52:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\salin\AppData\Local\Temp\tmprw1915lq\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.54454s
INFO:tensorflow:Finished evaluation at 2020-03-15-22:52:34
INFO:tensorflow:Saving dict for global step 500: ac

## 3. Recurrent Neural Networks model (RNN)

In [28]:
# SimpleRNN model
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=1))
# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(GRU(256, return_sequences=True))
# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(SimpleRNN(128))
model.add(Dense(1))

# View inside the network
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 1)           10        
_________________________________________________________________
gru (GRU)                    (None, None, 256)         198912    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 248,331
Trainable params: 248,331
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')

# Fit model
model.fit(X, Y, validation_data=(XX, YY), epochs=3, batch_size=64)

Train on 1410 samples, validate on 157 samples
Epoch 1/3
1410/1410 [==============================] - 2s 2ms/sample - loss: 0.0710 - val_loss: 0.0248
Epoch 2/3
1410/1410 [==============================] - 1s 401us/sample - loss: 0.0666 - val_loss: 0.0250
Epoch 3/3
1410/1410 [==============================] - 1s 407us/sample - loss: 0.0700 - val_loss: 0.0252


In [30]:
# Evaluate model
scores = model.evaluate(XX, YY, verbose=0)
print("Accuracy: %.2f%%" % (scores*100))

Accuracy: 2.52%


***
**Summary:**
1. The accuracy of test data from **Simple Neural Networks model** is **97.45%** which is very good.
2. The accuracy of test data from **Deep Neural Networks model** is also **97.45%** which is very good.
3. However, the accuracy of test data from **Recurrent Neural Networks model** is only **2.52%** which is very low.

Note: This is a very simple model without any advanced techniques. The result is just in a preliminary investigation stage and we can't make any conclusion if one of the model is out perform another model.
***

***
**Solicit Feedback:** 

- For **DNN**, I used **DNNClassifier** from tensorflow since it's the simplest one. The accuracy rate on test data is pretty high. How do we test for the overfitting?
- For **RNN**, I used **SimpleRNN** from tensorflow since it's the simplest one. I didn't include the timestamp from the label data set at all. My question is, how do we include the time as an input since RNN is about recurring.
***